In [1]:
import torch
from torch import nn
import utils

In [2]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU()
    )

In [3]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    # 全局平均池化层
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

In [4]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [5]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, resize=224)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-05-30 23:58:43]
epoch: 1/10, loss 2.304, train acc 0.104, test acc 0.119
epoch: 2/10, loss 2.084, train acc 0.230, test acc 0.434
epoch: 3/10, loss 1.405, train acc 0.499, test acc 0.595
epoch: 4/10, loss 1.226, train acc 0.566, test acc 0.569
epoch: 5/10, loss 0.736, train acc 0.721, test acc 0.746
epoch: 6/10, loss 0.577, train acc 0.785, test acc 0.753
epoch: 7/10, loss 0.500, train acc 0.818, test acc 0.837
epoch: 8/10, loss 0.448, train acc 0.835, test acc 0.843
epoch: 9/10, loss 0.418, train acc 0.846, test acc 0.863
epoch: 10/10, loss 0.387, train acc 0.858, test acc 0.863
*** 715.2 examples/sec on cuda:0 - [0:00:13:58], all: [0:00:18:48] ***
